In [ ]:
! pip install Flask


In [ ]:
! pip install import_ipynb

In [ ]:
! pip install pymysql

In [ ]:
! pip install WTForms

In [ ]:
! pip install mysqlclient

In [ ]:
! pip install flask-mysqldb


In [ ]:
! pip install flask-login

In [ ]:
! pip freeze

In [43]:
! pip install Flask-Bcrypt #lok

In [44]:
from flask import Flask, request, render_template, redirect, url_for, jsonify, flash
import import_ipynb
import forms, hashlib 
from flask_mysqldb import MySQL
from werkzeug.security import generate_password_hash, check_password_hash
from flask_bcrypt import Bcrypt #lok

In [10]:
# intilize flask application
app = Flask(__name__)
bcrypt = Bcrypt(app) #lok

In [11]:
app.config["SECRET_KEY"]="Hola como etas muy bein y tu?"
app.config['MYSQL_HOST']="localhost"
app.config["MYSQL_USER"]="root"
app.config["MYSQL_PASSWORD"]=""
app.config["MYSQL_DB"]="schoolapp"

In [12]:
mysql = MySQL(app)

In [13]:
@app.route("/")
def home():
    return redirect(url_for("login"))

In [14]:
@app.route("/login",methods=["GET","POST"])
def login():
    if request.method=="POST":
        login_form=forms.LoginForm(request.form)
        if login_form.validate():
            user_data=request.form
            email=user_data["email"]
            password=user_data["password"]
            sql="select * from users where email=%s"
            cursor=mysql.connection.cursor()
            cursor.execute(sql,(email,))
            account = cursor.fetchone()
            if account and bcrypt.check_password_hash(account[5], password):
                return "account exist"
            else :
                return "Invalid login credentials"
            cursor.close()
            return jsonify(login_form.data)
        else :
            return jsonify({"errors": login_form.errors}),400 
    else :
        return render_template("auth/user.html") 

In [15]:

@app.route("/register",methods=["GET","POST"]) #lok
def register(): #lol
    if request.method=="POST": #lol
        print(request.form) #lo
        register_form=forms.RegistrationForm(request.form) #ol
        print(register_form.data) #lo
        if register_form.validate(): #lo
            # return jsonify(register_form.data) #er
            user_data= register_form.data #gdg//
            first_name=user_data["first_name"] #dfd
            last_name=user_data["last_name"] #dfg
            middle_name=user_data["middle_name"] #dff
            email=user_data["email"] #ggfd
            password=user_data["password"] #gffd
            phone_number=user_data["phone_number"] #fgd
            # hash=hashlib.sha1(hash.encode()) #gfh
            # password=hash.digest() #ghf
            password =  bcrypt.generate_password_hash(password) #gh
            cursor=mysql.connection.cursor() #gf
            cursor.execute("select email from users where email=%s",(email,)) #gf
            account = cursor.fetchone() #gh
            if account: #fg
                return "account shown" #fg
            else: lol
                cursor.execute("insert into users ( `first_name`, `last_name`, `middle_name`, `email`, `password`, `gender`, `mobile_number`, `whatsapp_number`, `address`, `city`, `state`, `country`, `profile_picture`, `role`, `status`)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(first_name,last_name,middle_name,email,password,'',phone_number,'','','','','','','users','active')) #pop
                mysql.connection.commit() #kip
                cursor.close() #lok
                return redirect(url_for("login")) #lo
        else :
            # return jsonify({"errors" : register_form.errors }), 400 #hi
            flash({"errors" : register_form.errors }) #hu
            # return redirect(request.referrer) hi
            return jsonify({"errors" : register_form.errors }) #hi
    else :
        return render_template("auth/registartion.html") #hello



In [ ]:
if __name__== "__main__":
    app.run(debug=False)